<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-09-11 12:04:26


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=1e28dc5778e336e8168dcf0faab5f86417705a3e9a45b187f3854fc2b7565573
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/'

mypfs = pd.read_csv(path_mypf + 'myPortfolioStocks.csv')
mypps = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

df_mypf = pd.merge(mypfs, mypps, on="Symbol")


def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

In [5]:
df_mypf = df_mypf[df_mypf['InPortfolio'].isnull()]
cols = ['Symbol', 'Criteria', 'Strategy', 'LatestQtr', 'Target','StarStock', 'BizCheck', 'FinCheck', 'TechCheck', 'Conviction', 'Category', 'Stock']
df_mypf = df_mypf[cols]
all_prospects = get_stock_ids(df_mypf)

In [6]:
def print_distribution():

  # star stocks
  print('Star Prospects:')
  print(df_mypf['StarStock'].value_counts())
  print('-------')

  # v40/v40n stocks
  print('V40/V40N Prospects:')
  print(df_mypf['Criteria'].value_counts())
  print('-------')

  # latest quarter
  print('Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())
  print('-------')

  # star + v40/v40n stocks
  print('Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())
  print('-------')

print_distribution()

Star Prospects:
0    70
1    67
Name: StarStock, dtype: int64
-------
V40/V40N Prospects:
V40N    29
V40     26
AR       5
V200     2
SS       1
VR       1
Name: Criteria, dtype: int64
-------
Latest Qtr:
1    125
0     12
Name: LatestQtr, dtype: int64
-------
Star + V40/V40N Stocks:
1    30
0    25
Name: StarStock, dtype: int64
-------


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

from google.colab import data_table

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [9]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),0)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Curr_PE'] = current_pe
      tmp['Curr_ROCE%'] = current_roce
      tmp['Curr_ROE%'] = current_roe
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

In [10]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-08-11 00:00:00+05:30,19428.0,18281.0,6.28,^NSEI,19979.0,48.0
2023-08-21 00:00:00+05:30,19394.0,18321.0,5.85,^NSEI,19979.0,47.0
2023-08-28 00:00:00+05:30,19306.0,18352.0,5.20,^NSEI,19979.0,44.0
2023-09-04 00:00:00+05:30,19529.0,18380.0,6.25,^NSEI,19979.0,56.0
2023-09-11 00:00:00+05:30,19942.0,18416.0,8.28,^NSEI,19979.0,70.0


In [11]:
stock_ids = df_common_features['Stock'].values
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [12]:
df_prospects = pd.merge(df_common_features, df_mypf, on='Stock')

In [13]:
#df_common_features['Discount%PE'] = round((df_common_features['MedianPE']-df_common_features['Curr_PE'])*100/(df_common_features['MedianPE']),0)
df_prospects['Target'] = np.where(df_prospects['Strategy']== 'BTT', df_prospects["Target"], df_prospects['Max'])
df_prospects['Upside%Target'] = round((df_prospects['Target']-df_prospects['Close'])*100/(df_prospects['Close']),0)
df_prospects.drop([ '200_SMA', 'Symbol', 'Target', 'Max', 'Close'], axis=1, inplace=True)
df_prospects = df_prospects.set_index('Stock')

In [14]:
# top 10 prospects to invest from BTT stocks
df_prospects[df_prospects['Strategy'] == 'BTT'].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
ASIANPAINT.NS,6.33,48.0,67.0,35.2,27.7,V40,BTT,1,1,1,1,1.0,H,PAINTS,31.0
BAJFINANCE.NS,12.41,59.0,36.2,11.8,23.5,V40,BTT,1,1,1,1,1.0,H,FINANCE,24.0
ABBOTINDIA.NS,5.44,44.0,47.2,41.1,31.6,V40,BTT,1,1,1,1,NaN,M,PHARMA,20.0
AKZOINDIA.NS,16.20,52.0,34.2,34.7,26.1,V40,BTT,1,1,1,1,NaN,M,NaN,20.0
UJJIVANSFB.NS,53.48,65.0,1000.0,1000.0,1000.0,V40N,BTT,1,1,1,1,1.0,M,BANKS,8.0
COLPAL.NS,23.64,64.0,127.0,180.0,158.0,V40,BTT,1,1,0,1,1.0,H,FMCG,4.0


In [15]:
# prospects to invest from star + V40/V40N/VR + below 200 DMA
df_prospects[(df_prospects['Criteria'].notnull()) & (df_prospects['StarStock'] == 1) & (df_prospects['Dev%_200'] < 5)].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
VINATIORGA.NS,4.53,78.0,46.0,30.4,22.60,V40N,NaN,1,1,1,NaN,NaN,M,NaN,18.0
RELIANCE.NS,0.06,48.0,25.8,10.0,8.94,V40,ATH,1,1,1,1,1.0,H,REFINERIES,15.0
DABUR.NS,3.17,56.0,58.7,23.3,19.50,V40,NaN,1,1,0,1,1.0,H,FMCG,13.0
HINDUNILVR.NS,-1.42,47.0,57.4,26.6,20.50,V40,NaN,1,1,0,NaN,1.0,H,FMCG,9.0


In [16]:
# top 10 prospects to invest from star stocks
df_prospects[df_prospects['StarStock'] == 1].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
ADANIGREEN.NS,-12.72,56.0,131.00,7.81,22.8,NaN,NaN,1,1,0,1,0.0,L,NaN,194.0
BLUESTARCO.NS,13.39,66.0,53.50,24.50,23.4,AR,ATH,1,1,1,1,1.0,H,AC,88.0
TATAELXSI.NS,8.68,58.0,451.00,60.00,39.3,V40N,NaN,1,1,1,NaN,NaN,M,NaN,43.0
SBICARD.NS,5.11,55.0,36.20,13.00,25.7,NaN,NaN,1,1,0,NaN,1.0,M,NaN,33.0
ASIANPAINT.NS,6.33,48.0,67.00,35.20,27.7,V40,BTT,1,1,1,1,1.0,H,PAINTS,31.0
KANSAINER.NS,17.34,61.0,50.50,14.50,10.7,V40N,NaN,1,1,1,NaN,NaN,M,NaN,29.0
MUTHOOTFIN.NS,17.11,54.0,13.70,12.10,17.9,NaN,NaN,1,1,0,NaN,0.0,L,FINANCE,28.0
TASTYBITE.NS,42.80,51.0,1000.00,1000.00,1000.0,NaN,NaN,1,1,1,NaN,NaN,M,NaN,24.0
BAJFINANCE.NS,12.41,59.0,36.20,11.80,23.5,V40,BTT,1,1,1,1,1.0,H,FINANCE,24.0


In [17]:
# top 10 prospects to invest from  V40/V40N/VR
df_prospects[df_prospects['Criteria'].notnull()].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
JCHAC.NS,16.31,64.0,1000.0,1000.00,1000.00,V40N,NaN,1,0,1,NaN,NaN,M,NaN,148.0
TANLA.NS,29.71,50.0,28.9,37.90,31.20,AR,ATH,0,0,1,1,0.0,M,IT,95.0
BLUESTARCO.NS,13.39,66.0,53.5,24.50,23.40,AR,ATH,1,1,1,1,1.0,H,AC,88.0
LALPATHLAB.NS,5.59,49.0,70.6,18.50,15.00,V40N,NaN,1,0,1,NaN,NaN,M,NaN,86.0
SFL.NS,-0.15,56.0,53.8,13.60,12.50,V40N,NaN,1,0,1,NaN,NaN,M,NaN,71.0
SYMPHONY.NS,-4.16,63.0,56.2,14.80,13.50,V40N,NaN,1,0,1,NaN,NaN,M,NaN,62.0
AVANTIFEED.NS,15.79,68.0,18.5,20.70,13.40,V200,NaN,1,0,1,1,NaN,M,NaN,61.0
DREAMFOLKS.NS,4.01,35.0,36.9,76.90,60.20,AR,ATH,0,0,0,1,0.0,L,MISC,60.0
WHIRLPOOL.NS,19.21,86.0,103.0,8.78,6.37,V40,NaN,1,0,0,NaN,1.0,M,NaN,56.0


In [18]:
# top 10 prospects to invest below 200 DMA
df_prospects[df_prospects['Dev%_200'] < 0].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
ADANIGREEN.NS,-12.72,56.0,131.0,7.81,22.8,NaN,NaN,1,1,0,1,0.0,L,NaN,194.0
ALKYLAMINE.NS,-1.13,61.0,67.6,41.30,41.4,NaN,NaN,0,0,1,1,0.0,M,CHEMICALS,84.0
SFL.NS,-0.15,56.0,53.8,13.60,12.5,V40N,NaN,1,0,1,NaN,NaN,M,NaN,71.0
SYMPHONY.NS,-4.16,63.0,56.2,14.80,13.5,V40N,NaN,1,0,1,NaN,NaN,M,NaN,62.0
PFIZER.NS,-2.07,46.0,115.0,27.20,19.8,V40,NaN,1,0,0,NaN,0.0,L,PHARMA,56.0
UPL.NS,-10.26,58.0,17.0,15.70,13.4,NaN,NaN,1,0,0,NaN,NaN,L,NaN,33.0
HONAUT.NS,-0.78,31.0,1000.0,1000.00,1000.0,V40N,NaN,1,0,1,NaN,NaN,M,NaN,25.0
VEDL.NS,-9.12,40.0,11.8,23.80,20.4,NaN,NaN,1,0,0,NaN,NaN,L,NaN,24.0
HINDUNILVR.NS,-1.42,47.0,57.4,26.60,20.5,V40,NaN,1,1,0,NaN,1.0,H,FMCG,9.0


In [19]:
# star + v40/v40n prospects (for breakout)

tmp_df = df_prospects[df_prospects['StarStock'] == 1]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=True)

qualified stocks: 14


,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
3MINDIA.NS,32.86,70.0,74.8,31.40,23.4,V40N,NaN,1,1,1,NaN,NaN,M,NaN,0.0
AXISBANK.NS,8.13,59.0,13.4,6.16,14.5,V40,NaN,1,1,0,1,1.0,M,BANKS,0.0
HCLTECH.NS,16.72,79.0,23.0,28.30,23.0,V40,NaN,1,1,0,NaN,1.0,H,IT,0.0
MARICO.NS,13.26,65.0,55.8,43.00,35.8,V40,NaN,1,1,0,NaN,1.0,M,FMCG,0.0
MCDOWELL-N.NS,22.04,67.0,62.5,19.90,17.5,V40N,NaN,1,1,0,NaN,0.0,L,BREWERIES,0.0
OFSS.NS,30.94,80.0,21.0,35.10,24.8,V40N,NaN,1,1,1,NaN,1.0,M,IT,0.0
TITAN.NS,19.45,75.0,88.7,25.10,30.8,V40,NaN,1,1,0,NaN,1.0,H,JEWELLERY,0.0
ANGELONE.NS,38.01,66.0,17.2,44.00,47.1,V40N,NaN,1,1,1,1,NaN,M,FINANCE,1.0
ICICIBANK.NS,7.91,57.0,18.2,6.35,17.2,V40,NaN,1,1,0,NaN,1.0,H,BANKS,1.0


In [20]:
# prospects for interactive analysis

data_table.DataTable(df_prospects, include_index=True)

,Dev%_200,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,
3MINDIA.NS,32.86,70.0,74.80,31.40,23.40,V40N,NaN,1,1,1,NaN,NaN,M,NaN,0.0
ABBOTINDIA.NS,5.44,44.0,47.20,41.10,31.60,V40,BTT,1,1,1,1,NaN,M,PHARMA,20.0
ACC.NS,4.37,62.0,45.20,10.00,7.06,NaN,NaN,0,0,0,1,0.0,L,CEMENT,32.0
ADANIENT.NS,1.26,57.0,105.00,10.10,9.63,NaN,NaN,1,0,0,1,1.0,M,NaN,62.0
ADANIGREEN.NS,-12.72,56.0,131.00,7.81,22.80,NaN,NaN,1,1,0,1,0.0,L,NaN,194.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VOLTAS.NS,11.59,78.0,91.00,9.74,4.42,NaN,NaN,1,0,0,NaN,0.0,L,AC,45.0
WHIRLPOOL.NS,19.21,86.0,103.00,8.78,6.37,V40,NaN,1,0,0,NaN,1.0,M,NaN,56.0
WSTCSTPAPR.NS,36.89,73.0,4.77,61.80,44.80,VR,ATH,0,0,0,NaN,NaN,L,PAPER,3.0


In [21]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [22]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,RSI_14
Date,,,,,,
2023-08-11 00:00:00+05:30,701.0,610.0,14.87,BERGEPAINT.NS,854.0,59.0
2023-08-21 00:00:00+05:30,706.0,613.0,15.14,BERGEPAINT.NS,854.0,61.0
2023-08-28 00:00:00+05:30,709.0,616.0,15.10,BERGEPAINT.NS,854.0,60.0
2023-09-04 00:00:00+05:30,708.0,619.0,14.35,BERGEPAINT.NS,854.0,54.0
2023-09-11 00:00:00+05:30,723.0,622.0,16.23,BERGEPAINT.NS,854.0,62.0
